# LangSmith 사용하기

## 환경 셋팅

In [10]:
# API key 로딩
from dotenv import load_dotenv
import os

load_dotenv(override=True)
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
LANGSMITH_API_KEY = os.environ["LANGSMITH_API_KEY"]

from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

## 코드 실행

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from rich.console import Console
from rich.panel import Panel
import json

console = Console()

# 출력을 위한 JSON 스키마 정의
json_schema = """
{{
  "destination": "목적지",
  "duration": "기간",
  "overview": "여행 개요",
  "daily_plans": [
   {{
      "day": 1,
      "title": "제목",
      "morning": "오전 일정",
      "lunch": "점심 추천",
      "afternoon": "오후 일정",
      "dinner": "저녁 추천",
      "accommodation": "숙소 추천"
    }}
  ],
  "tips": ["팁1", "팁2"],
  "total_budget": "예상 총 비용"
}}
"""

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.5,
    model_kwargs={"response_format": {"type": "json_object"}}
)

# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", f"""당신은 {{place}} 베테랑 여행 가이드입니다.
고객 최적의 {{place}} {{travel}} 일정 수립을 도와줍니다.

다음의 JSON 형식으로 응답하세요.
{json_schema}"""),
    ("human", "해당 장소 {place}의 {travel} 일정에 맞는 여행 계획을 제안해주세요.")
])

# Json 출력 파서 정의
output_parser = JsonOutputParser()

# 체인 구성
chain = prompt | llm | output_parser

# llm 호출 - 자동으로 LangSmith에 기록됨!
input_data = {
    "place": "서울",
    "travel": "3일"
}

console.print("[bold cyan]여행 계획 생성 중...[/bold cyan]")
result = chain.invoke(input_data)

# 결과 출력
console.print(Panel(
    json.dumps(result, indent=2, ensure_ascii=False),
    title="[bold green]여행 계획[/bold green]",
    border_style="green"
))

여행 계획 생성 중...

╭─────────────────────────────────────────────────── 여행 계획 ───────────────────────────────────────────────────╮
│ {                                                                                                               │
│   "destination": "서울",                                                                                        │
│   "duration": "3일",                                                                                            │
│   "overview": "서울의 전통과 현대가 어우러진 매력적인 여행 일정입니다. 고궁 탐방, 맛있는 음식, 쇼핑과 문화      │
│ 체험을 포함한 알찬 계획입니다.",                                                                                │
│   "daily_plans": [                                                                                              │
│     {                                                                                                           │
│       "day": 1,                                                                                                 │
│       "title": "역사와 전통의 하루",                                                                            │
│       "morning": "경복궁 탐방 및 국립민속박물관 관람",                                                          │
│       "lunch": "북촌 한옥마을 근처의 전통 한정식",                                                              │
│       "afternoon": "북촌 한옥마을 산책 및 인사동 거리 탐방",                                                    │
│       "dinner": "인사동의 전통 찻집에서 저녁 및 차",                                                            │
│       "accommodation": "명동의 부티크 호텔"                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "day": 2,                                                                                                 │
│       "title": "현대 서울의 매력",                                                                              │
│       "morning": "남산서울타워 방문 및 케이블카 체험",                                                          │
│       "lunch": "명동의 유명한 길거리 음식",                                                                     │
│       "afternoon": "동대문 디자인 플라자(DDP) 탐방 및 쇼핑",                                                    │
│       "dinner": "홍대의 트렌디한 레스토랑에서 저녁",                                                            │
│       "accommodation": "홍대 인근 게스트하우스"                                                                 │
│     },                                                                                                          │
│     {                                                                                                           │
│       "day": 3,                                                                                                 │
│       "title": "문화와 자연의 조화",                                                                            │
│       "morning": "창덕궁과 후원 탐방",                                                                          │
│       "lunch": "종로의 유명한 비빔밥 집",                                                                       │
│       "afternoon": "서울숲에서 산책 및 자전거 대여",                                                            │
│       "dinner": "가로수길의 이탈리안 레스토랑",                                                                 │
│       "accommodation": "강남의 스타일리시한 호텔"                                                               │
│     }                                                                                                           │
│   ],                                                                                                            │
│   "tips": [                                                                                                     │
│     "대중교통 이용 시 T-money 카드 구매 추천",                                                         

# google gemini 모니터링하기

In [7]:
# 1. 필요한 패키지 설치
# pip install langchain-google-genai python-dotenv

import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from rich.console import Console
import json

load_dotenv(override=True)
console = Console()

# 2. 환경변수 설정 (.env 파일)
"""
GOOGLE_API_KEY=your-google-api-key
LANGCHAIN_TRACING_V2=true
LANGCHAIN_API_KEY=your-langsmith-api-key
LANGCHAIN_PROJECT=gemini-travel-planner
"""

# 3. JSON 스키마
json_schema = """
{{
  "destination": "목적지",
  "duration": "기간",
  "overview": "여행 개요",
  "daily_plans": [
   {{
      "day": 1,
      "title": "제목",
      "morning": "오전 일정",
      "lunch": "점심 추천",
      "afternoon": "오후 일정",
      "dinner": "저녁 추천",
      "accommodation": "숙소 추천"
    }}
  ],
  "tips": ["팁1", "팁2"],
  "total_budget": "예상 총 비용"
}}
"""

# 4. Gemini LLM 초기화 (OpenAI 대신)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # 또는 "gemini-1.5-pro"
    temperature=0.5,
    # JSON 모드는 Gemini에서도 지원됨
)

# 5. 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", f"""당신은 {{place}} 베테랑 여행 가이드입니다.
고객 최적의 {{place}} {{travel}} 일정 수립을 도와줍니다.

다음의 JSON 형식으로 응답하세요.
{json_schema}"""),
    ("human", "해당 장소 {place}의 {travel} 일정에 맞는 여행 계획을 제안해주세요.")
])

# 6. 출력 파서
output_parser = JsonOutputParser()

# 7. 체인 구성
chain = prompt | llm | output_parser

# 8. 실행 (자동으로 LangSmith에 추적됨!)
console.print("[bold cyan]Gemini로 여행 계획 생성 중...[/bold cyan]")

result = chain.invoke(
    {"place": "서울", "travel": "3일"},
    config={
        "metadata": {
            "model": "gemini-2.5-flash",
            "user_id": "user-123"
        },
        "tags": ["gemini", "travel", "production"]
    }
)

# 9. 결과 출력
console.print("[bold green]✓ 완료![/bold green]")
console.print(json.dumps(result, indent=2, ensure_ascii=False))

console.print("\n[bold yellow]LangSmith에서 확인:[/bold yellow]")
console.print("https://smith.langchain.com")

Gemini로 여행 계획 생성 중...

✓ 완료!

{
  "destination": "서울 (Seoul)",
  "duration": "3일",
  "overview": "서울의 찬란한 역사와 현대적인 매력을 동시에 경험하는 3일 여정입니다. 고궁의 아름다움 속에서 한국의 
전통을 느끼고, 트렌디한 거리에서 서울의 역동적인 에너지를 만끽하며, 맛있는 음식과 함께 오감을 만족시키는 특별한 
여행이 될 것입니다.",
  "daily_plans": [
    {
      "day": 1,
      "title": "서울의 심장을 걷다: 역사와 전통의 향기",
      "morning": "경복궁 방문 (한복 체험 추천) 및 근정전, 경회루 등 주요 전각 관람. 이어서 북촌 한옥마을 산책 
(가회동 31번지, 11번지 일대)하며 전통 한옥의 정취 느끼기.",
      "lunch": "삼청동/인사동 거리에서 한정식 또는 전통 떡갈비 정식. 골목골목 숨어있는 맛집을 찾아보는 재미.",
      "afternoon": "인사동 쌈지길과 전통찻집 골목 구경. 조계사 방문하여 도심 속 고즈넉한 사찰 분위기 경험. 
마지막으로 광장시장으로 이동하여 자유로운 시장 분위기 느끼기.",
      "dinner": "광장시장에서 빈대떡, 마약김밥, 떡볶이 등 길거리 음식 투어. 막걸리와 함께 현지 분위기 만끽.",
      "accommodation": "명동 또는 종로 지역 호텔 (접근성 좋고 다양한 숙소 선택 가능)"
    },
    {
      "day": 2,
      "title": "도심 속 스카이라인과 트렌드의 발견",
      "morning": "남산서울타워 케이블카 탑승 후 전망대에서 서울 전경 감상. 사랑의 자물쇠 남기기. 이어서 명동으로 
이동하여 자유 쇼핑 및 거리 구경.",
      "lunch": "명동 거리의 다양한 맛집 (명동 칼국수, 닭갈비, 돈까스 등) 중 취향에 맞는 식사.",
      "afternoon": "강남으로 이동하여 가로수길 (신사동) 산책하며 트렌디한 카페와 편집숍 구경. 이어서 코엑스 별마당 
도서관 방문하여 이색적인 공간 체험.",
      "dinner": "강남역 또는 신사동 일대에서 세련된 레스토랑 또는 퓨전 한식 / 강남 스타일의 고품격 한우/돼지고기 
BBQ.",
      "accommodation": "명동 또는 종로 지역 호텔 (어제와 동일하거나 다른 숙소 선택 가능)"
    },
    {
      "day": 3,
      "title": "청춘의 에너지와 예술, 그리고 여유로운 마무리",
      "morning": "홍대 거리 방문하여 자유로운 분위기 만끽 (버스킹 공연 관람, 벽화골목 구경). 이색적인 카페에서 
브런치 즐기기.",
      "lunch": "홍대 주변의 다양한 젊은 층 취향 저격 맛집 (파스타, 햄버거, 퓨전 일식 등) 또는 분위기 좋은 브런치 
카페.",
      "afternoon": "연남동 '연트럴파크' (경의선 숲길) 산책하며 여유로운 오후 보내기. 아기자기한 소품샵 구경 및 개성
있는 카페에서 휴식.",
      "dinner": "고객님의 취향에 맞춰, 첫날 못 가본 동네에서 즐기는 만찬 (예: 을지로 노포 골목에서 레트로 감성 
즐기기, 동대문 디자인 플라자(DDP) 근처 현대적인 레스토랑).",
      "accommodation": "출국 전이라면 공항 근처 숙소 또는 서울 시내 호텔 (개인 일정에 따라)"
    }
  ],
  "tips": [
    "**교통카드 'T-Money' 준비**: 지하철, 버스 등 대중교통 이용에 필수적이며 매우 편리합니다. 편의점에서 구매 및 
충전 가능합니다.",
    "**모바일 지도 앱 활용**: '네이버 지도' 또는 '카카오맵'을 다운로드하여 사용하시면 길 찾기가 매우 용이합니다. 
대중교통 정보도 정확합니다.",
    "**환전 및 결제**: 주요 관광지 및 상점에서는 카드 결제가 보편적이지만, 전통시장이나 길거리 음식점에서는 현금이 
편리할 수 있습니다. 주요 은행이나 환전소에서 환전하세요.",
    "**계절별 복장**: 서울은 사계절이 뚜렷합니다. 봄/가을은 일교차가 크고, 여름은 습하고 더우며, 겨울은 매우 
춥습니다. 방문 시기에 맞춰 적절한 복장을 준비하세요.",
    "**무료 와이파이**: 서울 시내 곳곳에 'SEOUL_Public_Wifi' 등 무료 와이파이가 잘 되어있습니다. 데이터 로밍이나 
포켓 와이파이 없이도 어느 정도 이용 가능합니다.",
    "**비상 연락처**: 주한 대사관 전화번호, 경찰(112), 응급의료(119) 등 비상 연락처를 미리 알아두세요.",
    "**유연한 일정**: 위에 제안된 일정은 추천 코스이며, 고객님의 관심사나 컨디션에 따라 언제든지 변경하거나 조절할 
수 있습니다."
  ],
  "total_budget": "예상 총 비용: 1인 기준 약 70만원 ~ 120만원 (숙소 등급, 식사 취향, 쇼핑 여부에 따라 변동)"
}

LangSmith에서 확인:

https://smith.langchain.com